In [1]:
import pandas as pd
import scanpy as sc
import anndata
import time
import graph_tool.all as gt
import pickle

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:40: RuntimeWarning: Error importing draw module, proceeding nevertheless: /usr/lib/python3/dist-packages/cairo/_cairo.cpython-37m-x86_64-linux-gnu.so: undefined symbol: cairo_tee_surface_index
  warnings.warn(msg, RuntimeWarning)


In [2]:
%matplotlib inline
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
import anndata
adata=sc.read('adata_10x/adata_sc_10x.h5ad')
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
df_significativo= pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20) #### df con i 20 geni significativi per gruppo
cell_annotation= pd.read_table('../sc_mixology/data/csv/sc_10x_5cl.metadata.csv.gz', compression="gzip", sep=",", index_col=0)
adata.obs.loc[:, "cell_line"] = cell_annotation.loc[adata.obs_names, "cell_line"]

/home/morelli/anaconda3/envs/scnsbm/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead.
  """


In [4]:
g = pickle.load( open( "adata_10x/10x_graph_g.pkl", "rb" ) ) 

In [5]:
a=time.time()
state=gt.minimize_nested_blockmodel_dl(g, deg_corr=False)
b=time.time()
b-a

29.540435314178467

In [14]:
state.get_levels()[2]

<BlockState object with 4 blocks (4 nonempty), for graph <Graph object, undirected, with 13 vertices and 34 edges at 0x7faaac025d68>, at 0x7faaac035a90>

In [13]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
0,62,0,4,1,188
1,0,0,127,0,0
2,0,176,7,0,0
3,0,0,0,295,0


In [32]:
bs = state.get_bs()                   
bs += [np.zeros(1)] * (10 - len(bs))

estate = state.copy(bs=bs, sampling=True)
a=time.time()
gt.mcmc_equilibrate(estate, epsilon=0, wait=1000,nbreaks=2,mcmc_args=dict(niter=10,beta=1,c=1))
b=time.time()
print(b-a)
estate

127.40937662124634


<NestedBlockState object, with base <BlockState object with 65 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5976 edges at 0x7faab256cdd8>, at 0x7faaa5b490b8>, and 10 levels of sizes [(860, 63), (63, 15), (15, 7), (7, 6), (6, 5), (5, 4), (4, 3), (3, 3), (3, 2), (2, 2)] at 0x7faaa6b312e8>

In [40]:
pd.Categorical(list(estate.project_level(4).get_blocks()))

[1, 5, 5, 1, 1, ..., 4, 4, 0, 4, 4]
Length: 860
Categories (5, int64): [0, 1, 3, 4, 5]

In [36]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(estate.project_level(4).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
0,0,0,0,295,0
1,1,0,4,1,188
3,0,0,56,0,0
4,61,176,7,0,0
5,0,0,71,0,0


In [67]:
bs = estate.get_bs()                   
bs += [np.zeros(1)] * (10 - len(bs))

estate1 = estate.copy(bs=bs, sampling=True)
a=time.time()
gt.mcmc_equilibrate(estate1, epsilon=0, wait=1000,nbreaks=2,mcmc_args=dict(niter=10,beta=1,c=1))
b=time.time()
print(b-a)
estate1

72.9628598690033


<NestedBlockState object, with base <BlockState object with 67 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5976 edges at 0x7faab256cdd8>, at 0x7faaa6ba62e8>, and 10 levels of sizes [(860, 63), (63, 15), (15, 7), (7, 4), (4, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 2)] at 0x7faaa59fe748>

In [69]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(estate1.project_level(3).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
3,61,0,71,0,0
5,0,176,7,295,0
7,0,0,56,0,0
9,1,0,4,1,188


In [4]:
states_100=[]
for i in range (100):
    i=gt.minimize_nested_blockmodel_dl(g, deg_corr=False)
    states_100.append(i)

In [6]:
pickle.dump(states_100, open('adata_10x/10x_100_states.pkl','wb'), 2)

In [4]:
states_100=pickle.load(open('adata_10x/10x_100_states.pkl','rb'))

In [4]:
states_100

[<NestedBlockState object, with base <BlockState object with 67 blocks (67 nonempty), for graph <Graph object, undirected, with 860 vertices and 5976 edges at 0x7f728dfa48d0>, at 0x7f7272e5a390>, and 5 levels of sizes [(860, 67), (67, 14), (14, 5), (5, 2), (2, 1)] at 0x7f724888c110>,
 <NestedBlockState object, with base <BlockState object with 64 blocks (64 nonempty), for graph <Graph object, undirected, with 860 vertices and 5976 edges at 0x7f728dfa48d0>, at 0x7f7272df6510>, and 5 levels of sizes [(860, 64), (64, 14), (14, 4), (4, 2), (2, 1)] at 0x7f724888c550>,
 <NestedBlockState object, with base <BlockState object with 60 blocks (60 nonempty), for graph <Graph object, undirected, with 860 vertices and 5976 edges at 0x7f728dfa48d0>, at 0x7f7272dfef90>, and 5 levels of sizes [(860, 60), (60, 14), (14, 4), (4, 2), (2, 1)] at 0x7f7272df6d10>,
 <NestedBlockState object, with base <BlockState object with 70 blocks (70 nonempty), for graph <Graph object, undirected, with 860 vertices and 

In [8]:
states_100[0].print_summary()

l: 0, N: 860, B: 67
l: 1, N: 67, B: 14
l: 2, N: 14, B: 5
l: 3, N: 5, B: 2
l: 4, N: 2, B: 1


In [11]:
max(states_100[0].get_bs()[2])

4

In [11]:
states_100[0]

<NestedBlockState object, with base <BlockState object with 67 blocks (67 nonempty), for graph <Graph object, undirected, with 860 vertices and 5976 edges at 0x7f91e11f4090>, at 0x7f92077dab10>, and 5 levels of sizes [(860, 67), (67, 14), (14, 5), (5, 2), (2, 1)] at 0x7f920783f850>

In [34]:
x=max(states_100[0].get_bs()[2])
x

4

In [5]:
bss=[]
for i in range(100):
    x=max(states_100[i].get_bs()[2])
    bss.append(x)

In [17]:
bss[0:6]

[4, 3, 3, 4, 4, 2]

In [40]:
groups_5=[]
groups_4=[]
groups_3=[]
groups_6=[]
for i in bss:
    if i==4:
        groups_5.append(i)
    elif i==3:
        groups_4.append(i)
    elif i==2:
        groups_3.append(i)
    else:
        groups_6.append(i)
    

In [42]:
len(groups_5)

48

In [43]:
len(groups_4)

49

In [44]:
len(groups_3)

2

In [45]:
len(groups_6)

1

In [17]:
import anndata

In [18]:
adata=sc.read('adata_10x/adata_sc_10x.h5ad')
adata

AnnData object with n_obs × n_vars = 860 × 1596 
    obs: 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'leiden'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [19]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')

In [20]:
df_significativo= pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20) #### df con i 20 geni significativi per gruppo

In [21]:
cell_annotation= pd.read_table('../sc_mixology/data/csv/sc_10x_5cl.metadata.csv.gz', compression="gzip", sep=",", index_col=0)

In [22]:
adata.obs.loc[:, "cell_line"] = cell_annotation.loc[adata.obs_names, "cell_line"]
adata.obs.columns

Index(['n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase',
       'leiden', 'cell_line'],
      dtype='object')

In [23]:
for i in range (len(states_100)):
    adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(states_100[i].project_level(2).get_blocks()))
    figure(i)
    sc.pl.umap(adata, color="NSBM_test")

... storing 'cell_line' as categorical
/usr/lib/python3/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [24]:
for i in range (len(states_100)):
    adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(states_100[i].project_level(2).get_blocks()))
    print(pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line))

cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
4            61     0      0      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2             0     0      0    295       0
3            61   176      7      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0            62     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127    

cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
4            61     0      0      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
4            61     0      0      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1            61   176      7      0       0
2             0     0      0    295       0
3             0     0    127      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4    

cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
4            61     0      0      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0            62     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
4            61     0      0      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4    

cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0            62     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2             0   176      7      0       0
3             0     0      0    295       0
4            61     0      0      0       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127      0       0
2            61   176      7      0       0
3             0     0      0    295       0
cell_line  A549  H838  H1975  H2228  HCC827
NSBM_test                                  
0             1     0      4      1     188
1             0     0    127    

### NSBM con description length

In [5]:
test1= gt.minimize_nested_blockmodel_dl(g, deg_corr=False)

In [6]:
test2=gt.minimize_nested_blockmodel_dl(g, deg_corr=False)

In [11]:
test3=gt.minimize_nested_blockmodel_dl(g, deg_corr=False)

In [16]:
pickle.dump(test1, open('adata_10x/state_test1.pkl', 'wb'),2)
pickle.dump(test2, open('adata_10x/state_test2.pkl', 'wb'),2)
pickle.dump(test3, open('adata_10x/state_test3.pkl', 'wb'),2)

In [50]:
test3.entropy()


19492.83981692386

In [49]:
test2.entropy()

19511.151648651357

In [51]:
test1.entropy()

19512.737415460844

### Hashing 

In [52]:
hashing=[]
for i in range(100):
    x=hash(states_100[i])
    hashing.append(x)

In [56]:
len(hashing)

100

##### non va bene hash perchè genera un codice diverso per ognuno dei 100 grafi, al posto di unire grafi con le stesse caratteristiche. devo provare a farlo in un altro modo

In [76]:
max(states_100[0].get_bs()[2])==max(states_100[3].get_bs()[2])

True

In [77]:
max(states_100[0].get_bs()[2])

4

In [111]:
state_levels=[]
for i in range(100):
    levels=[]
    for j in range(5):
        x=max(states_100[i].get_bs()[j])
        levels.append(x)
    state_levels.append(levels)

In [112]:
state_levels[0:6]

[[66, 13, 4, 1, 0],
 [63, 13, 3, 1, 0],
 [59, 13, 3, 1, 0],
 [69, 15, 4, 1, 0],
 [71, 13, 4, 1, 0],
 [45, 8, 2, 1, 0]]

In [113]:
state_type=[]
state_test=state_levels
for i in state_levels:
    state_test.remove(i)
    livelli=[]
    livelli.append(i)
    for j in state_test:
        if i==j:
            state_levels.remove(j)
            livelli.append(j)
    state_type.append(livelli)

In [114]:
len(state_type)

37

In [115]:
state_type

[[[66, 13, 4, 1, 0], [66, 13, 4, 1, 0], [66, 13, 4, 1, 0]],
 [[59, 13, 3, 1, 0]],
 [[71, 13, 4, 1, 0]],
 [[73, 15, 3, 1, 0]],
 [[58, 12, 3, 1, 0], [58, 12, 3, 1, 0]],
 [[67, 14, 3, 1, 0]],
 [[64, 14, 4, 1, 0], [64, 14, 4, 1, 0]],
 [[63, 13, 4, 1, 0], [63, 13, 4, 1, 0], [63, 13, 4, 1, 0], [63, 13, 4, 1, 0]],
 [[65, 15, 4, 1, 0], [65, 15, 4, 1, 0]],
 [[59, 12, 3, 1, 0], [59, 12, 3, 1, 0], [59, 12, 3, 1, 0]],
 [[63, 14, 3, 1, 0], [63, 14, 3, 1, 0]],
 [[52, 13, 3, 1, 0]],
 [[67, 15, 5, 2, 1]],
 [[64, 11, 3, 1, 0], [64, 11, 3, 1, 0]],
 [[67, 17, 4, 1, 0], [67, 17, 4, 1, 0]],
 [[51, 12, 3, 1, 0]],
 [[53, 11, 2, 1, 0]],
 [[60, 12, 3, 1, 0]],
 [[69, 13, 4, 1, 0], [69, 13, 4, 1, 0], [69, 13, 4, 1, 0]],
 [[64, 13, 4, 1, 0], [64, 13, 4, 1, 0]],
 [[57, 13, 3, 1, 0]],
 [[64, 14, 3, 1, 0]],
 [[55, 11, 3, 1, 0]],
 [[64, 13, 3, 1, 0], [64, 13, 3, 1, 0], [64, 13, 3, 1, 0], [64, 13, 3, 1, 0]],
 [[54, 12, 3, 1, 0]],
 [[54, 13, 3, 1, 0], [54, 13, 3, 1, 0]],
 [[66, 15, 4, 1, 0], [66, 15, 4, 1, 0]],
 [[65, 

In [116]:
len(state_levels)

34

In [117]:
state_levels

[[63, 13, 3, 1, 0],
 [69, 15, 4, 1, 0],
 [45, 8, 2, 1, 0],
 [68, 14, 4, 1, 0],
 [56, 11, 3, 1, 0],
 [69, 15, 4, 1, 0],
 [54, 11, 3, 1, 0],
 [63, 12, 4, 1, 0],
 [57, 12, 3, 1, 0],
 [65, 14, 4, 1, 0],
 [56, 14, 4, 1, 0],
 [63, 15, 4, 1, 0],
 [50, 11, 3, 1, 0],
 [54, 10, 3, 1, 0],
 [67, 16, 4, 1, 0],
 [69, 14, 3, 1, 0],
 [72, 15, 4, 1, 0],
 [69, 15, 3, 1, 0],
 [65, 14, 3, 1, 0],
 [74, 18, 4, 1, 0],
 [68, 11, 3, 1, 0],
 [51, 11, 3, 1, 0],
 [59, 11, 3, 1, 0],
 [52, 10, 3, 1, 0],
 [68, 15, 4, 1, 0],
 [67, 15, 4, 1, 0],
 [65, 16, 4, 1, 0],
 [56, 12, 3, 1, 0],
 [63, 15, 3, 1, 0],
 [65, 13, 3, 1, 0],
 [67, 14, 4, 1, 0],
 [69, 14, 4, 1, 0],
 [63, 16, 4, 1, 0],
 [65, 13, 3, 1, 0]]

##### in questo modo però non riesco a discernere se la suddivisione è avvenuta correttamente